In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
df_clients = pd.read_csv("../Data/Raw/df_final_experiment_clients.txt")
df_demo = pd.read_csv("../Data/Raw/df_final_demo.txt")
df_web1 = pd.read_csv("../Data/Raw/df_final_web_data_pt_1.txt")
df_web2 = pd.read_csv("../Data/Raw/df_final_web_data_pt_2.txt")